In [ ]:
import os
import sys

project_root = os.path.dirname(os.getcwd())
sys.path.append(f"{project_root}/src")
sys.path.append(f"{project_root}/third_party")

In [ ]:
import hydra
from hydra import compose, initialize

hydra.core.global_hydra.GlobalHydra.instance().clear()
initialize(config_path="../config/", version_base=None)
gpt2_cfg = compose(config_name="gpt2")
gpt2_cfg.project_root = project_root


In [ ]:
import ray

text_data_file_path =  gpt2_cfg.dataset[0].path




In [ ]:


ds = ray.data.read_text(text_data_file_path)

print(ds.schema())